# Discriminative use case: Sentiment Analysis

In this notebook, we will explore how we can perform classical ML task such as sentiment analysis using LLM.

### Use case description

Sentiment analysis is a common NLP task that involves classifying text into a pre-defined sentiment. For example, with Miyagi, we can classify a user's stock portfolio into a predefined categories based on current market sentiment, which can be determined using a Bing search.

> Note - To improve reliability of the model output, set the temperature to 0 and use top_k sampling. This will ensure that the model always generates the same output for the same input.

#### 1. Import Semantic Kernel dependencies

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230711.7-preview"
#r "nuget: Microsoft.SemanticKernel.Planning.StepwisePlanner, 0.17.230711.7-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.17.230711.7-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

#### 2. Initialize kernel

In [ ]:
var builder = new KernelBuilder();

var (_, model, azureEndpoint, apiKey, org, bingApiKey) = Settings.LoadFromFile();
builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);

// TODO: Add embedding service for RaG

IKernel kernel = builder.Build();

#### 3. Create a Semantic Function (Prompt Template)

In [ ]:
string skPrompt = @"
This is a chat between a classification system that responds in valid JSON and human looking to rate their stock portfolio. Given a portfolio in valid JSON, respond back a sentiment in JSON for the attribute: sentiment. To determine this attribute, consider Web results from Bing and propietary data below.

Example:
""stocks"": [
        {
            ""symbol"": ""MSFT"",
            ""allocation"": 0.3,
            ""sentiment"": ""positive""
        },
        {
            ""symbol"": ""ACN"",
            ""allocation"": 0.1,
            ""sentiment"": ""negative""
        },
        {
            ""symbol"": ""JPM"",
            ""allocation"": 0.3,
            ""sentiment"": ""neutral""
        },
        {
            ""symbol"": ""PEP"",
            ""allocation"": 0.3,
            ""sentiment"": ""neutral""
        }
    ]

Rate my portfolio:

{{$portfolio}}


Context: 
Web Results: {{$web}}
";

// create semantic function
var sentimentClassifier = kernel.CreateSemanticFunction(
    skPrompt,
    skillName: "SentimentClassifier",
    functionName: "SentimentClassifer",
    description: "Classify sentiment of a stock portfolio",
    maxTokens: 1000, temperature: 0, topP: 0.5);
Console.WriteLine(textToSQL.ToString());

#### 3. Create Context variables

In [ ]:
var portfolio = @"
    ""stocks"": [
        {
            ""symbol"": ""AAPL"",
            ""allocation"": 0.2,
            ""sentiment"": """"
        },
        {
            ""symbol"": ""NVDA"",
            ""allocation"": 0.3,
            ""sentiment"": """"
        },
        {
            ""symbol"": ""TSLA"",
            ""allocation"": 0.2,
            ""sentiment"": """"
        },
        {
            ""symbol"": ""T"",
            ""allocation"": 0.3,
            ""sentiment"": """"
        }
    ]
";

// context variables
var context = kernel.CreateNewContext();
context["portfolio"] = portfolio;

#### 4. Generate SQL from natural language

In [ ]:
var question = "Determine the sentiment of my stock portfolio based on what the internet says about the stocks in my portfolio.";

var bingConnector = new BingConnector(bingApiKey);
var webSearchEngineSkill = new WebSearchEngineSkill(bingConnector);

kernel.ImportSkill(sentimentClassifier, "SentimentClassifier");
kernel.ImportSkill(webSearchEngineSkill, "WebSearch");

var config = new Microsoft.SemanticKernel.Planning.Stepwise.StepwisePlannerConfig();
config.MinIterationTimeMs = 1500;
config.MaxTokens = 4000;

StepwisePlanner planner = new(kernel, config);
var plan = planner.CreatePlan(question);

var result = await plan.InvokeAsync(kernel.CreateNewContext());
Console.WriteLine("Result: " + result);
if (result.Variables.TryGetValue("stepCount", out string? stepCount))
{
    Console.WriteLine("Steps Taken: " + stepCount);
}

if (result.Variables.TryGetValue("skillCount", out string? skillCount))
{
    Console.WriteLine("Skills Used: " + skillCount);
}
